In [1]:
import AdventOfCode

In [2]:
input <- dayString 9

In [3]:
import Text.Read (readMaybe)
import Data.Maybe (mapMaybe, catMaybes)

[playing, final] = mapMaybe (readMaybe :: String -> Maybe Int) $ words input

In [4]:
import qualified Data.IntMap.Strict as Map

In [5]:
data Zipper a = Zipper
    { zipperLeft  :: [a]
    , zipperFocus :: a
    , zipperRight :: [a]
    } deriving (Eq, Show)

moveLeft  (Zipper [] c r) = let
    (c':l) = reverse (c:r)
    in Zipper l c' []
moveLeft  (Zipper (x:xs) c r) = Zipper xs    x (c:r)

moveRight (Zipper l c []) = let
    (c':r) = reverse (c:l)
    in Zipper [] c' r
moveRight (Zipper l c (x:xs)) = Zipper (c:l) x xs

insertAt e (Zipper l c r) = Zipper l e (c:r)

deleteAt (Zipper l c r) = case r of
    [] -> case reverse l of
        [] -> error "empty Zipper"
        (x:xs) -> Zipper [] x xs
    (x:xs) -> Zipper l x xs

fromList (x:xs) = Zipper [] x xs

moveLeft $ Zipper [] 1 [2,3]
moveRight $ Zipper [2,1] 3 []
deleteAt $ Zipper [] 1 [2,3]

Zipper {zipperLeft = [2,1], zipperFocus = 3, zipperRight = []}

Zipper {zipperLeft = [], zipperFocus = 1, zipperRight = [2,3]}

Zipper {zipperLeft = [], zipperFocus = 2, zipperRight = [3]}

In [6]:
data GameState
    = GameState
    { placedMarbles :: Zipper Int
    , currentMarble :: Int
    , currentPlayer :: Int
    , players :: Map.IntMap Int
    , totalPlayers :: Int
    } deriving (Eq, Show)

In [7]:
play :: GameState -> GameState
play gs = let
    nextPlayer = (currentPlayer gs + 1) `mod` (totalPlayers gs)
    current = currentMarble gs
    (placed, score) = place current (placedMarbles gs)
    players' = Map.adjust (+score) nextPlayer (players gs)
    in GameState placed (current+1) nextPlayer players' (totalPlayers gs)

moveLeftN 1 z = moveLeft z
moveLeftN n z = moveLeftN (n-1) (moveLeft z)

moveRightN 1 z = moveRight z
moveRightN n z = moveRightN (n-1) (moveRight z)

place :: Int -> Zipper Int -> (Zipper Int, Int)
place marble placed
    | marble `mod` 23 == 0 = let
        moved = moveLeftN 7 placed
        remove = zipperFocus moved
        placed' = deleteAt moved
        in (placed', remove + marble)
    | otherwise = let
        placed' = insertAt marble (moveRightN 2 placed)
        in (placed', 0)

In [8]:
toList (Zipper l c r) = reverse l ++ (c:r)

In [9]:
import Data.List
import Data.Function (on)

initGame p f = GameState (fromList [0]) 1 0 (Map.fromList $ zip [0..p] (repeat 0)) p

loop g f = if (currentMarble g > f) then maximumBy (compare `on` snd) (Map.toList (players g)) else loop (play g) f

In [10]:
loop (initGame 9 25) 25
loop (initGame 10 1618) 1618
loop (initGame 13 7999) 7999

(5,32)

(0,8317)

(12,146373)

In [11]:
loop (initGame playing final) final

(326,393229)

In [12]:
loop (initGame playing (final*100)) (final*100)

(191,3273405195)